In [1]:
import cv2
import numpy as np
from tensorflow import keras
from collections import deque

In [2]:
model = keras.models.load_model('summer2021_tune_model_final')

In [3]:
labels=[ "Prob(B1+B2_Complete)", 
        "Prob(CA1_Incomplete)", "Prob(CA1_Complete)", 
        "Prob(CA2_Incomplete)", "Prob(CA2_Complete)",
        "Prob(B3_Incomplete)","Prob(B3_Complete)",
        "Prob(B4_Incomplete)","Prob(B4_Complete+B6_Incomplete)",
        "Prob(B5_Incomplete)","Prob(B5_Complete)",
        "Prob(B6_Complete+CO1_Incomplete)",
        "Prob(CA4_Incomplete)","Prob(CA4_Complete+CO1_Incomplete)",
        "Prob(CO1_Complete)",
        "Prob(E2_Incomplete)","Prob(E2_Complete)"]

In [5]:
cap = cv2.VideoCapture('OneDrive/Desktop/New Data/Overall Video/Scene (14).mp4')
#cap = cv2.VideoCapture('sequence.mp4')
IMG_SIZE=50 
if (cap.isOpened()== False):
    print("Error opening video stream or file")
    
window_size = 10   
ave = deque(maxlen = window_size)
predict_list = []
prob_list = []
while True:
    ret, frame = cap.read()
    gray =cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.resize(cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE),(720,960))
    #frame = cv2.resize(cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE),(720,960))
    #frame = cv2.resize(frame,(720,960))
    img = cv2.resize(gray,(IMG_SIZE,IMG_SIZE))
    X=img.reshape(-1,IMG_SIZE,IMG_SIZE,1)
    font = cv2.FONT_HERSHEY_SIMPLEX
    predictions = model.predict(X)
    ave.append(predictions)
    if len(ave) == window_size:
        ave_np= np.array(ave)
        ave_mean = ave_np.mean(axis = 0)
        #indices = np.argsort(-ave_mean)
        predict_1 = labels[2]
        predict_2 = 'Prob(Rest)'
        prob_1 = ave_mean[0][2]
        prob_2 = 1-prob_1
        a1 = np.array([predict_1, predict_2])
        a2 = np.array([prob_1,prob_2]) 
        cv2.putText(frame, str(predict_1) + str(prob_1),
                (10, 50), 
                font, 0.6, 
                (255, 0, 0), 
                2, 
                cv2.LINE_4)
        cv2.putText(frame, str(predict_2) + str(prob_2),
                (10, 80), 
                font, 0.6, 
                (0, 0, 255), 
                2, 
                cv2.LINE_4)
        cv2.imshow('frame',frame)
        predict_list.append(a1)
        prob_list.append(a2)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [ ]:
import pandas as pd
df=pd.DataFrame(predict_list, columns = ["Predict_1","Predict_2"])
dg=pd.DataFrame(prob_list, columns = ["Prob_1","Prob_2"])
writer = pd.ExcelWriter('result.xlsx', engine='xlsxwriter')
df.to_excel(writer, header = True, startrow=0, startcol=0,float_format='%.5f')
dg.to_excel(writer, header = True, index=False, startrow=0, startcol=3,float_format='%.5f')
writer.save()

In [ ]:
cap = cv2.VideoCapture('sequence.mp4')

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (720,960))

IMG_SIZE=50 
if (cap.isOpened()== False):
    print("Error opening video stream or file")
    
window_size = 15   
ave = deque(maxlen = window_size)
while True:
    ret, frame = cap.read()
    gray =cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #frame = cv2.resize(cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE),(720,960))
    #frame = cv2.resize(cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE),(720,960))
    frame = cv2.resize(frame,(720,960))
    img = cv2.resize(gray,(IMG_SIZE,IMG_SIZE))
    X=img.reshape(-1,IMG_SIZE,IMG_SIZE,1)
    font = cv2.FONT_HERSHEY_SIMPLEX
    predictions = model.predict(X)
    ave.append(predictions)
    if len(ave) == window_size:
        ave_np= np.array(ave)
        ave_mean = ave_np.mean(axis = 0)
        indices = np.argsort(-ave_mean)
        predict_1 = labels[indices[0][0]]
        predict_2 = labels[indices[0][1]]
        predict_3 = labels[indices[0][2]]
        
        cv2.putText(frame, str(predict_1) + str(ave_mean[0][indices[0][0]]),
                (10, 50), 
                font, 0.6, 
                (255, 0, 0), 
                2, 
                cv2.LINE_4)
        cv2.putText(frame, str(predict_2) + str(ave_mean[0][indices[0][1]]),
                (10, 80), 
                font, 0.6, 
                (0, 0, 255), 
                2, 
                cv2.LINE_4)
        cv2.putText(frame, str(predict_3) + str(ave_mean[0][indices[0][2]]),
                (10, 110), 
                font, 0.6, 
                (0, 0, 255), 
                2, 
                cv2.LINE_4)
        out.write(frame)
        cv2.imshow('frame',frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
cap.release()
out.release()
cv2.destroyAllWindows()